In [1]:
import os
import time
import numpy as np
import pandas as pd
import re

import keras
from keras import *
from keras import layers
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from keras.models import Model
from keras.preprocessing import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pickle

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
sms_df= pd.read_csv("trains.csv")

In [3]:
sms_df

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
...,...,...,...,...
10013,16925,Bernie Sanders Introduces Killer Mike At Coac...,"On April 16th, Senator Bernie Sanders, through...",1
10014,16926,TRUMP SUPPORTERS STORM MAXINE WATERS TOWN HALL...,This is great! Trump supporters did everything...,1
10015,16927,Trump adds nuance to pro-Israel approach ahead...,WASHINGTON/JERUSALEM (Reuters) - During his 20...,0
10016,16928,Top U.S. officials to testify in Trump-Russia ...,WASHINGTON (Reuters) - The U.S. House of Repre...,0


In [4]:
sms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10018 entries, 0 to 10017
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  10018 non-null  int64 
 1   title       9951 non-null   object
 2   text        10009 non-null  object
 3   label       10018 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 313.2+ KB


In [5]:
sms_df.isnull().sum()

Unnamed: 0     0
title         67
text           9
label          0
dtype: int64

In [6]:
sms_df=sms_df.dropna()

In [7]:
sms_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9942 entries, 0 to 10017
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  9942 non-null   int64 
 1   title       9942 non-null   object
 2   text        9942 non-null   object
 3   label       9942 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 388.4+ KB


In [8]:
sms_df['label'].unique()

array([1, 0], dtype=int64)

In [9]:
sms_df['label']= pd.to_numeric(sms_df['label'])

<ipython-input-9-5cccea66aba9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sms_df['label']= pd.to_numeric(sms_df['label'])


In [10]:
texts = sms_df['text']
labels = sms_df['label']

In [11]:
other_texts, test_texts, other_labels, test_labels  = train_test_split(texts, labels, test_size=0.1, random_state=302)
#Create validation sample
train_texts, valid_texts, train_labels, valid_labels  = train_test_split(other_texts, other_labels, test_size=0.2, random_state=302)

In [12]:
VOCABULARY_SIZE = 25000

In [13]:
tokenizer = Tokenizer(num_words=VOCABULARY_SIZE)
tokenizer.fit_on_texts(train_texts)

# Convert words into word ids
meanLength = np.mean([len(item.split(" ")) for item in train_texts])
MAX_SENTENCE_LENGTH = int(meanLength + 5) # we let a text go 10 words longer than the mean text length.

# Convert train, validation, and test text into lists with word ids
trainFeatures = tokenizer.texts_to_sequences(train_texts)
trainFeatures = pad_sequences(trainFeatures, MAX_SENTENCE_LENGTH, padding='post')
trainLabels = train_labels.values

validFeatures = tokenizer.texts_to_sequences(valid_texts)
validFeatures = pad_sequences(validFeatures, MAX_SENTENCE_LENGTH, padding='post')
validLabels = valid_labels.values

testFeatures = tokenizer.texts_to_sequences(test_texts)
testFeatures = pad_sequences(testFeatures, MAX_SENTENCE_LENGTH, padding='post')
testLabels = test_labels.values

In [14]:
MAX_SENTENCE_LENGTH

547

In [15]:
FILTERS_SIZE = 16
KERNEL_SIZE = 5

# Define embeddings dimensions (columns in matrix fed into CNN and nodes in hidden layer of built-in keras function)
EMBEDDINGS_DIM = 11

# Hyperparameters for model tuning
LEARNING_RATE = 0.001
BATCH_SIZE = 32
EPOCHS = 18

In [16]:
# Word CNN
model = Sequential()

# We use built-in keras funtion to generate embeddings. Another option is pre-trained embeddings with Word2vec or GloVe.
model.add(Embedding(input_dim=VOCABULARY_SIZE + 1, output_dim=EMBEDDINGS_DIM, input_length=len(trainFeatures[0])))
model.add(Conv1D(FILTERS_SIZE, KERNEL_SIZE, activation='relu'))
model.add(Dropout(0.5))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
            
optimizer = optimizers.Adam(lr=LEARNING_RATE)
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 547, 11)           275011    
_________________________________________________________________
conv1d (Conv1D)              (None, 543, 16)           896       
_________________________________________________________________
dropout (Dropout)            (None, 543, 16)           0         
_________________________________________________________________
global_max_pooling1d (Global (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 8)                 136       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9

In [17]:
history = model.fit(trainFeatures, trainLabels, validation_data = (validFeatures, validLabels), batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/18
224/224 [==============================] - 4s 16ms/step - loss: 0.6504 - accuracy: 0.6120 - val_loss: 0.5433 - val_accuracy: 0.8553
Epoch 2/18
224/224 [==============================] - 3s 15ms/step - loss: 0.4224 - accuracy: 0.8284 - val_loss: 0.3189 - val_accuracy: 0.8821
Epoch 3/18
224/224 [==============================] - 3s 15ms/step - loss: 0.2979 - accuracy: 0.8893 - val_loss: 0.2604 - val_accuracy: 0.9106
Epoch 4/18
224/224 [==============================] - 3s 15ms/step - loss: 0.2161 - accuracy: 0.9275 - val_loss: 0.2188 - val_accuracy: 0.9201
Epoch 5/18
224/224 [==============================] - 3s 15ms/step - loss: 0.1596 - accuracy: 0.9440 - val_loss: 0.2010 - val_accuracy: 0.9168
Epoch 6/18
224/224 [==============================] - 3s 15ms/step - loss: 0.1158 - accuracy: 0.9626 - val_loss: 0.1604 - val_accuracy: 0.9358
Epoch 7/18
224/224 [==============================] - 3s 15ms/step - loss: 0.0897 - accuracy: 0.9744 - val_loss: 0.1611 - val_accuracy: 0.9369

In [24]:
twt = ["WASHINGTON/JERUSALEM (Reuters) - During his 20"]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)

In [25]:
twt = pad_sequences(twt, maxlen=720, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=20,verbose = 2)[0]
print(sentiment)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

In [26]:
current_dir = os.getcwd()
output_dir = re.sub('Model and data', 'Flask application', current_dir)
os.chdir(output_dir)

In [27]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
model.save('cnn.h5')